In [2]:
import os, cv2
import pandas as pd
from PIL import Image
from collections import defaultdict
import json
from pathlib import Path
from argparse import ArgumentParser
import requests
import zipfile
import numpy as np
import shutil

In [3]:
base_path = 'd:\\Dev\\PYTHON\\S3\\'

In [19]:
imgPath = base_path + "ideo\\images\\"
labelPath = base_path + "ideo\\labels\\"
annotation_path = base_path + "labels.json"

In [5]:
def generateYOLO(filename, bbox):
    file = open(os.path.join(labelPath, filename+'.txt'), "w")
    file.writelines(bbox)
    file.close

In [6]:
def convert(size, box, img_id):
    if box[0] > size[0] or box[2] > size[0] or box[1] > size[1] or box[3] > size[1]:  
        ###wrong labelling, label coordinate is bigger than or overside the image size
        w = open(os.path.join('tagging_box_error'), 'a')
        w.write(img_id+'\n')
        w.close
        return [''] ###return nothing, give up this label
    dw = 1./(size[0])
    dh = 1./(size[1])
    xmax = abs(box[0]) + abs(box[2])
    ymax = abs(box[1]) + abs(box[3])
    if xmax > size[0]:  ###avoid the label box out of the image
        xmax = size[0]
    if ymax > size[1]:
        ymax = size[1]
    x = (abs(box[0]) + xmax)/2.0 - 1
    y = (abs(box[1]) + ymax)/2.0 - 1
    w = abs(box[2])
    h = abs(box[3])
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return [str(0), ' ', str(round(x, 6)),' ', str(round(y, 6)), ' ', str(round(w, 6)), ' ', str(round(h, 6)), '\n'] 
    ###the first one, index 0, is the class id and assuem "person" class is 0

In [7]:
annoFile = open(annotation_path)
data = json.load(annoFile)

In [9]:
annotations = data['tracking-annotations']

In [21]:
for lineID, line in enumerate(annotations):
    img_name = line['frame']
    img_id = os.path.splitext(img_name)[0]
    
    image_path = imgPath + img_name
    
    if os.path.exists(image_path):
        img = cv2.imread(image_path)
        (img_height, img_width, img_depth) = img.shape
        total_iid = len(line["annotations"]) 
        box_list = []
        for iid, infoBody in enumerate(line["annotations"]):
            box_list.extend(convert((img_width, img_height), [infoBody['']], img_id))
            # print("[Left over] {}/{} , {}: head:{} full:{} visible:{}".format(total_lines-lineID, total_iid-iid, img_id,\
            #         hbox, fbox, vbox)) #end of one label object loop 
        generateYOLO(img_id, box_list) #end of one image loop
    else:
        f = open(os.path.join('no_this_image_log'), 'a')  ###can't find the label file name in imgPath
        f.write(img_id+'\n')
        f.close

{'height': 109, 'width': 73, 'top': 586, 'left': 408, 'class-id': '0', 'label-category-attributes': {}, 'object-id': '8a9c5520-8eda-11ee-b714-17bb8c48c37b', 'object-name': 'human:1'}
{'height': 109, 'width': 73, 'top': 618, 'left': 431, 'class-id': '0', 'label-category-attributes': {}, 'object-id': '8a9c5520-8eda-11ee-b714-17bb8c48c37b', 'object-name': 'human:1'}
{'height': 109, 'width': 73, 'top': 623, 'left': 451, 'class-id': '0', 'label-category-attributes': {}, 'object-id': '8a9c5520-8eda-11ee-b714-17bb8c48c37b', 'object-name': 'human:1'}
{'height': 120, 'width': 72, 'top': 619, 'left': 449, 'class-id': '0', 'label-category-attributes': {}, 'object-id': '8a9c5520-8eda-11ee-b714-17bb8c48c37b', 'object-name': 'human:1'}
{'height': 120, 'width': 72, 'top': 619, 'left': 449, 'class-id': '0', 'label-category-attributes': {}, 'object-id': '8a9c5520-8eda-11ee-b714-17bb8c48c37b', 'object-name': 'human:1'}
{'height': 120, 'width': 72, 'top': 628, 'left': 439, 'class-id': '0', 'label-categor

KeyboardInterrupt: 